# You Only Look Once: Unified, Real-Time Object Detection  
- [paper](https://arxiv.org/pdf/1506.02640.pdf)

<img src = "https://github.com/fned49/blog/blob/main/YOLO/figure/figure1.png?raw=true">

- object detection을 single regression problem으로 재구성한다
- 하나의 CNN으로 bounding box와 class probability를 동시에 예측한다
- YOLO는 이미지 전체에 대해서 추론을 한다
- 전체 이미지를 살펴보며 이미지의 모양 뿐만 아니라, class에 대한 contextual information을 encoding한다
- object를 일반화할 수 있는 representation을 학습한다

<img src = "https://github.com/fned49/blog/blob/main/YOLO/figure/figure2.png?raw=true">

- input image를 $S\times S$ grid로 나눈다
- object의 center가 특정 grid cell에 속한다면, 해당 grid cell은 object를 탐지하는 역할을 한다
- **confidence** = $Pr\left(Object\right) * IOU_{pred}^{truth}$
- object의 center가 특정 grid cell에 속하지 않으면, 해당 grid cell의 confidence score는 0이다
- bounding box = {$x$, $y$, $w$, $h$, confidence score}
- 각 grid cell은 $C$ conditional class probability $Pr\left(Class_i\vert Object\right)$를 예측한다
- box의 갯수에 상관없이 grid cell 당 class probability의 one set을 예측한다   
- test time에 우리는 conditional class probability와 각 box confidence를 다음과 같이 곱한다  
$$Pr\left(Class_i\vert Object\right) * Pr\left(Object\right) * IOU_{pred}^{truth} = Pr\left(Class_i\right) * IOU_{pred}^{truth}$$  

<img src = "https://github.com/fned49/blog/blob/main/YOLO/figure/figure3.png?raw=true">

- network의 convolutional layer는 image의 feature를 추출하며 fc layer는 probability와 coordinate를 예측한다
- 24개의 convolutional layer, 2개의 fc layer
- $1\times 1$ conv layer 다음에 $3\times 3$ conv layer
- final output은 $7\times 7\times 30$ tensor

### Training

- conv layer는 ImageNet-1K dataset으로 pretrain을 진행했다
- 작은 object를 탐지하기 위해 image size를 $224\times 224$부터 $448\times 448$로 점차 늘려갔다  
- output layer는 class probability와 box coordinate를 예측하는데 이때 0~1 normalizaed된 값을 사용한다  
- finaly layer에는 linear activation function을 사용하고, 그 외에는 leaky ReLU activation function을 사용한다  
- SSE(sum-squared error)를 최적화를 한다 -> 이는 최적화하기 편리하지만 AP를 maximizing 하기에는 어려움이 있다  
  
보통 이미지에서 대부분의 grid cell은 object를 포함하지 않는다. 이는 no object의 gradient가 object의 gradient를 압도해버리는 문제가 생긴다.\
이를 개선하기 위해 box coordinate의 weight는 증가시키고 confidence의 weight는 감소시킨다.

- $\lambda_{coord}=5$, $\lambda_{noobj}=0.5$  

SSE에서 large box와 small box에 대한 error의 weight는 동일하다.\
그래서 small box보다 large box의 편차가 덜 중요하다는 것을 loss function에 반영해야 한다.\
이를 부분적으로 해결하기 위해 directly width, height 대신 square root를 씌운 값을 예측한다.

- loss function:
$$\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(x_i-\hat{x}_i\right)^2 + \left(y_i-\hat{y}_i\right)^2\right]$$  
$$+\lambda_{coord}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left[\left(\sqrt{w_i}-\sqrt{\hat{w}_i}\right)^2 + \left(\sqrt{h_i}-\sqrt{\hat{h}_i}\right)^2\right]$$  
$$+\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{obj}\left(C_i-\hat{C}_i\right)^2$$  
$$+\lambda_{noobj}\sum_{i=0}^{S^2}\sum_{j=0}^{B}\mathcal{1}_{ij}^{noobj}\left(C_i-\hat{C}_i\right)^2$$  
$$+\sum_{i=0}^{S^2}\mathcal{1}_{i}^{obj}\sum_{c\in classes}\left(p_i\left(c\right)-\hat{p}_i\left(c\right)\right)$$  
- $\mathcal{1}_i^{obj}$는 $i$번째 cell에 object가 존재하는지, $\mathcal{1}_{ij}^{obj}$는 $i$번째 cell, $j$번째 box에 object가 존재하는지 나타낸다  
- hyperparameter로 batch size는 64, momentum 0.9 그리고 decay는 0.0005를 사용한다  
- learning rate schedule로 처음 75 epoch까지는 $10^{-2}$로 학습하고 그 이후 30 epoch까지는 $10^{-3}$, 마지막 30 epoch 동안 $10^{-4}$로 학습한다  
- overfitting을 피하기 위해 우리는 dropout과 data augmentation을 적용했다  
- dropout rate는 0.5를 사용하며 first fc layer 이후에 적용한다  
- data augmentation으로 random scaling with scale 1.2, random HSV color space를 적용한다  